In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import BayesianGaussianMixture
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import RobustScaler,PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb


# Load the dataset
data = pd.read_csv("D:\Ki6\Phân tích chuỗi tgian\EDA\Gia SMP va SMPcap 2021(Giá thị trường SMP).csv", encoding='latin-1')

data.isnull().sum()
# Verify the remaining columns are all numeric
print(data.dtypes)


Ngay;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48    object
dtype: object


In [2]:
data.head()

,Ngay;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48
0,01/01/2021;964.4;964.4;964.4;964.4;964.4;964.4...
1,01/02/2021;1019.7;1019.7;1019.7;1019.7;1019.7;...
2,01/03/2021;988.4;988.4;988.4;988.4;988.4;988.4...
3,01/04/2021;1002.0;1002.0;1002.0;1002.0;1002.0;...
4,01/05/2021;1061.5;1061.5;1061.5;1061.5;1061.5;...


EDA 

In [4]:
#Biểu đồ trực quan hóa dữ liệu theo ngày
data.set_index('Ngay')
plt.plot(data.index,data['10'])
plt.title('Time Series Data with col 10')
plt.xlabel('Date')
plt.ylabel('7')
plt.show()


KeyError: "None of ['Ngày'] are in the columns"

In [20]:
plt.figure(figsize=(10,5))
plt.plot(data['6'],color = 'red',label = '6')
plt.plot(data['7'],color = 'blue',label = '7')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Comparison of 6 and 7')
plt.legend()
plt.show()

KeyError: '6'

<Figure size 1000x500 with 0 Axes>

BGM+PCA

In [ ]:
feats = ['6', '7']
transformer = PowerTransformer()
X=transformer.fit_transform(data[feats])

In [ ]:
pca = PCA(n_components=2,random_state=1)
pca.fit(X)
PCA_ds = pd.DataFrame(pca.transform(data[feats]), columns=(["col1","col2"]))
PCA_ds.describe().T

In [ ]:
BGM = BayesianGaussianMixture(n_components=7,covariance_type='full',random_state=1,n_init=15)
# fit model and predict clusters
preds = BGM.fit_predict(X)

#Adding the Clusters feature to the orignal dataframe.
data["Clusters"]= preds

In [ ]:
pp=BGM.predict_proba(X)# Calcualting the probabilities of each prediction
df_new=pd.DataFrame(X,columns=feats) 
df_new[[f'predict_proba_{i}' for i in range(7)]]=pp # creating new dataframe columns of probabilites 
df_new['preds']=preds
df_new['predict_proba']=np.max(pp,axis=1)
df_new['predict']=np.argmax(pp,axis=1)
    
train_index=np.array([])
for n in range(7):
    n_inx=df_new[(df_new.preds==n) & (df_new.predict_proba > 0.68)].index
    train_index = np.concatenate((train_index, n_inx))

In [ ]:
X_new=df_new.loc[train_index][feats]
y=df_new.loc[train_index]['preds']

params_lgb = {'learning_rate': 0.06,'objective': 'multiclass','boosting': 'gbdt','n_jobs': -1,'verbosity': -1, 'num_classes':7} 

model_list=[]

gkf = StratifiedKFold(11)
for fold, (train_idx, valid_idx) in enumerate(gkf.split(X_new,y)):   

    tr_dataset = lgb.Dataset(X_new.iloc[train_idx],y.iloc[train_idx],feature_name = feats)
    vl_dataset = lgb.Dataset(X_new.iloc[valid_idx],y.iloc[valid_idx],feature_name = feats)
    
    model = lgb.train(params = params_lgb, 
                train_set = tr_dataset, 
                valid_sets =  vl_dataset, 
                num_boost_round = 5000, 
                callbacks=[ lgb.early_stopping(stopping_rounds=300, verbose=False), lgb.log_evaluation(period=200)])  
    
    model_list.append(model)

ARIMA

In [ ]:
lgb_preds=0
for model in model_list:
    lgb_preds+=model.predict(df_new[feats])

In [ ]:
labels=np.argmax(lgb_preds,axis=1)

In [ ]:
print('ADF Statistic : %f' %adfuller(data['6'])[0])
print('p-values: %f'%adfuller(data['6'])[1])

In [ ]:
plot_acf(data['7'],lags =20)
plt.title('Autocorrelation Funtion')
plt.show()

plot_pacf(data['7'],lags =20)
plt.title('Partial Autocorrelation')
plt.show()

In [ ]:
model = ARIMA(data['7'],order =(1,1,1))
model = model.fit()

plt.figure(figsize = (12,6))
residuals = model.resid
plt.plot(residuals)
plt.xlabel('residuals')
plt.ylabel('date')
plt.title('Residuals')
plt.show()

In [ ]:
print('ADF Statistic : %f' %adfuller(residuals)[0])
print('p-values: %f'%adfuller(residuals)[1])

In [ ]:
col_index = 6
r_t = np.log(df.iloc[:, col_index] / df.iloc[:, col_index].shift(1)).values

mean = np.nanmean(r_t)
r_t[0]=mean
r_t[:6]

result = adfuller(r_t)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
    
plt.figure(figsize = (8, 6))
ax1 = plot_acf(r_t)

In [ ]:
plt.figure(figsize = (8, 6))
ax2 = plot_pacf(r_t)

In [ ]:
model_arima = ARIMA(r_t, order=(2, 0, 2))
model_fit = model_arima.fit()
print(model_fit.summary())